In [ ]:
import numpy as np

In [ ]:
# USAGE

# python predict_video.py --model model/activity_gpu.model --label-bin model/lb.pickle --input example_clips/cyclone_movie.mp4 --output output/cyclon_movie_output.avi --size 10

# import the necessary packages
from tensorflow.keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2
from twilio.rest import Client
import csv
import pandas as pd
import numpy as np
import datetime


# load the trained model and label binarizer from disk
print("[INFO] loading model and label binarizer...")
model = load_model("model")
lb = pickle.loads(open("model/lb.pickle", "rb").read())

# initialize the image mean for mean subtraction along with the
# predictions queue
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=30)

# initialize the video stream, pointer to output video file, and
# frame dimensions
vs = cv2.VideoCapture("example_clips/LeftTest.MOV")
writer = None
(W, H) = (None, None)
prelabel = ''
ok = 'Normal'
fi_label = []
framecount = 0

frames = []
shots = []
player = []

playerNo = round(np.random.rand()*100)

forehandCount = 0
backhandCount = 0

In [ ]:
while True:
	# read the next frame from the file
	(grabbed, frame) = vs.read()

	# if the frame was not grabbed, then we have reached the end
	# of the stream
	if not grabbed:
		break

	# if the frame dimensions are empty, grab them
	if W is None or H is None:
		(H, W) = frame.shape[:2]
	framecount = framecount+1
	# clone the output frame, then convert it from BGR to RGB
	# ordering, resize the frame to a fixed 224x224, and then
	# perform mean subtraction
#	cv2.imshow("Output", frame)
	output = frame.copy()
	frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	frame = cv2.resize(frame, (224, 224)).astype("float32")
	frame -= mean
#	cv2.imshow("Output", frame)

	# make predictions on the frame and then update the predictions
	# queue
	preds = model.predict(np.expand_dims(frame, axis=0))[0]
#	print('Preds = :', preds)
	
#	total = (preds[0]+ preds[1]+preds[2] + preds[3]+ preds[4]+preds[5])
#	maximum = max(preds)
#	rest = total - maximum
    
#	diff = (.8*maximum) - (.1*rest)
#	print('Difference of prob ', diff)
#	th = 100
#	if diff > .60:
#		th = diff
#	print('Old threshold = ', th)
    
    
	prediction = preds.argmax(axis=0)
	Q.append(preds)

	# perform prediction averaging over the current history of
	# previous predictions
	results = np.array(Q).mean(axis=0)
	print('Results = ', results)
	maxprob = np.max(results)
	print('Maximun Probability = ', maxprob)
	i = np.argmax(results)
	print("i = ", i)
	label = lb[i]

	rest = 1 - maxprob
    
	diff = (maxprob) - (rest)
	#print('Difference of prob ', diff)
	th = 100
	if diff > .80:
		th = diff
      
	frames.append(framecount)
	player.append(playerNo)
	shots.append(i)
        
        
	print(label, ':' , prelabel)   
	if framecount > 10:
		if label != prelabel:
			if (i==0):
				backhandCount += 1
			else:
				forehandCount += 1
    
	text = "{} - {:.2f}%".format((label), maxprob * 100)
	#text = ""
	text = text + "Backhand count : " + str(backhandCount) + "\n"
	text = text + "Forehand count : " + str(forehandCount)
	cv2.putText(frame, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 5)


	prelabel = label    



	# check if the video writer is None
	if writer is None:
		# initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		#writer = cv2.VideoWriter(args["output"] + datetime.datetime.now().strftime('%b-%d-%Y_%H%M'), fourcc, 30, (W, H), True)
		writer = cv2.VideoWriter( "output/"+ datetime.datetime.now().strftime('%b_%d_%Y_%H%M'+'.MOV'), fourcc, 30, (W, H), True)
		

# 	# write the output frame to disk
	writer.write(frame)

# 	# show the output image
	cv2.imshow("Output", frame)
	key = cv2.waitKey(1) & 0xFF

	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break
print('Frame count', framecount)
#print('Count label', fi_label)
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()
